# Logistic Regression

Let's see an example of how to run a logistic regression with Python and Spark! This is documentation example, we will quickly run through this and then show a more realistic example, afterwards, you will have another consulting project!

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregdoc').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
# Load data
data = spark.read.format("libsvm").load("/FileStore/tables/sample_libsvm_data.txt")

lr = LogisticRegression()

# Holdout sample
train_data, test_data = data.randomSplit([0.7,0.3])

# Fit the model
lrModel = lr.fit(train_data)

dataSummary = lrModel.summary

In [5]:
data.groupBy('label').count().show()

+-----+-----+
label|count|
+-----+-----+
 0.0| 43|
 1.0| 57|
+-----+-----+

In [6]:
data.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows

In [7]:
dataSummary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[98,99,100,1...|[24.3763295267624...|[0.99999999997408...| 0.0|
 0.0|(692,[121,122,123...|[22.0630218574259...|[0.99999999973809...| 0.0|
 0.0|(692,[122,123,124...|[19.8324343152307...|[0.99999999756284...| 0.0|
 0.0|(692,[123,124,125...|[30.9881725126846...|[0.99999999999996...| 0.0|
 0.0|(692,[123,124,125...|[22.8476894875255...|[0.99999999988049...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [8]:
# Usually would do this on a separate test set!
predictionAndLabels = lrModel.evaluate(test_data)

In [9]:
predictionAndLabels.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[95,96,97,12...|[22.3065193171642...|[0.99999999979469...| 0.0|
 0.0|(692,[100,101,102...|[10.0383822066854...|[0.99995631151082...| 0.0|
 0.0|(692,[122,123,148...|[20.3300328124440...|[0.99999999851823...| 0.0|
 0.0|(692,[123,124,125...|[30.5055872804397...|[0.99999999999994...| 0.0|
 0.0|(692,[124,125,126...|[31.5502818358234...|[0.99999999999998...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [10]:
predictionAndLabels = predictionAndLabels.predictions.select('label','prediction')

In [11]:
predictionAndLabels.show(5)

+-----+----------+
label|prediction|
+-----+----------+
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
+-----+----------+
only showing top 5 rows

## Evaluators

Evaluators will be a very important part of our pipline when working with Machine Learning, let's see some basics for Logistic Regression, useful links:

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [14]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

In [15]:
# For multiclass
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')

In [16]:
acc = evaluator.evaluate(predictionAndLabels)

In [17]:
acc

Out[71]: 1.0

Okay let's move on see some more examples!